In [ ]:
import pandas as pd
from surprise import SVD, Dataset, Reader
import tkinter as tk
from tkinter import messagebox

# Load the MovieLens dataset
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

# Prepare the data for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Train the SVD algorithm
trainset = data.build_full_trainset()
svd = SVD()
svd.fit(trainset)

# Function to get top N movie recommendations for a user
def get_top_n_recommendations(user_id, n=10):
    user_movies = ratings[ratings['userId'] == user_id]
    watched_movie_ids = user_movies['movieId'].unique()
    all_movie_ids = movies['movieId'].unique()
    recommendations = []
    for movie_id in all_movie_ids:
        if movie_id not in watched_movie_ids:
            pred = svd.predict(user_id, movie_id)
            recommendations.append((movie_id, pred.est))
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_n_recommendations = recommendations[:n]
    return [(movies[movies['movieId'] == movie_id]['title'].values[0], rating) for movie_id, rating in top_n_recommendations]

# Function to display recommendations
def show_recommendations():
    try:
        user_id = int(user_id_entry.get())
        if user_id < 1 or user_id > ratings['userId'].max():
            raise ValueError("User ID out of range")
        recommendations = get_top_n_recommendations(user_id)
        result_text = "Top 10 movie recommendations for you:\n"
        for title, rating in recommendations:
            result_text += f"{title} (predicted rating: {rating:.2f})\n"
        recommendations_text.set(result_text)
    except ValueError as e:
        messagebox.showerror("Error", str(e))

# Create the main window
root = tk.Tk()
root.title("Movie Recommendation System")

# Create and place widgets
tk.Label(root, text="Enter your user ID").pack(pady=5)
user_id_entry = tk.Entry(root)
user_id_entry.pack(pady=5)

tk.Button(root, text="Get Recommendations", command=show_recommendations).pack(pady=5)

recommendations_text = tk.StringVar()
recommendations_label = tk.Label(root, textvariable=recommendations_text, justify=tk.LEFT)
recommendations_label.pack(pady=5)

# Start the Tkinter event loop
root.mainloop()
